In [11]:
import utils 
import csv 
import pandas as pd

text_files, json_files = utils.return_file_path()
df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

for text_file, json_file in zip(text_files, json_files):
    output = "" 
    start_idx = 0 
    
    answer = ("prime" if "prime" in text_file else "composite")
    data_type = json_file.rstrip(".json").lstrip("../json_sampled/")
    querys = utils.return_json_data(json_file) 
    
    for query in querys:
        query_list = list(query.values())
        for i in range(len(query_list)-2):
            output, start_idx = utils.return_output(text_file, start_idx)
            if i >= 4:
                continue
            result = utils.return_PrimeOrComposite(output, i)
            
            # print(query["number"], f"Q{i+1}", output, result, query["answer"])
            df.loc[str(query["number"])+f"_Q{i+1}"] = (f"Q{i+1}", output, result, query["answer"], data_type ,query["number"])
df

,Qustion,Output,Predict,Answer,Difficulty,Number
10_Q1,Q1,"\nNo, 10 is not a prime number. It can be divi...",composite,composite,2_digit_composite,10
10_Q2,Q2,"\nNo, 10 is not a prime number because it is d...",composite,composite,2_digit_composite,10
10_Q3,Q3,"\nNo, 10 is not a composite number. A composit...",prime,composite,2_digit_composite,10
10_Q4,Q4,"\nNo, 10 is not a composite number. A composit...",prime,composite,2_digit_composite,10
12_Q1,Q1,"\nNo, 12 is not a prime number. It is divisibl...",composite,composite,2_digit_composite,12
...,...,...,...,...,...,...
2939_Q4,Q4,"\nTo determine if 2939 is a composite number, ...",prime,prime,4_digit_prime,2939
6521_Q1,Q1,"\nYes, 6521 is a prime number.\n",prime,prime,4_digit_prime,6521
6521_Q2,Q2,"\nTo determine if 6521 is a prime number, we c...",composite,prime,4_digit_prime,6521
6521_Q3,Q3,"\nYes, 6521 is a composite number.\n",composite,prime,4_digit_prime,6521


In [12]:
df.groupby('Difficulty').count()

,Qustion,Output,Predict,Answer,Number
Difficulty,,,,,
2_digit_composite,276,276,276,276,276
2_digit_prime,84,84,84,84,84
3_digit_easy_composite,808,808,808,808,808
3_digit_medium_composite,224,224,224,224,224
3_digit_prime,572,572,572,572,572
4_digit_easy_composite,2000,2000,2000,2000,2000
4_digit_hard_composite,524,524,524,524,524
4_digit_medium_composite,2000,2000,2000,2000,2000
4_digit_prime,2000,2000,2000,2000,2000


In [13]:
df_idk = df[df["Predict"]=="idk"]
df_idk.to_csv("../csv/PaLM2_idk.csv")

In [14]:
import random 

random.seed(1024)

sampled_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    temp_df = df[df['Difficulty'] == Difficulty]
    number_list = list(temp_df['Number'].unique())

    if len(number_list) >= 25:
        number_list = random.sample(number_list, k=25)
    
    for number in number_list:    
        df_25 = temp_df[temp_df['Number']==number]
        sampled_df = pd.concat([sampled_df, df_25])

sampled_df = sampled_df.drop(["Qustion", "Answer", "Difficulty", "Number"], axis=1)
sampled_df['Verdict'] = 0
sampled_df.to_csv("../csv/chatgpt_Sampled.csv")

In [15]:
len(sampled_df[sampled_df['Predict']=='idk'])

4

In [16]:
import numpy as np

acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-9:] == "composite":
        continue
    acc = np.array([0, 0, 0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_prime,100.0,100.0,100.0,100.0
3_digit_prime,98.6,100.0,97.2,95.8
4_digit_prime,70.2,94.6,25.6,93.4


In [17]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
acc = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], dtype="float")
digit_len = np.array([0.0, 0.0, 0.0])
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue    
    digit = int(Difficulty[0])
    temp_df = df[df['Difficulty'] == Difficulty]
    digit_len[digit-2] += (len(temp_df) // 4)
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[digit-2][i] += 1
            
            
for i in range(3):
    acc[i] = acc[i] / digit_len[i] * 100
    acc_df.loc[f"{i+2}_digit_composite"] = acc[i]
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_composite,100.00,94.20,68.12,89.86
3_digit_composite,91.86,67.05,87.21,56.20
4_digit_composite,57.56,27.67,86.30,27.32


In [18]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite'])

,Q1,Q2,Q3,Q4
2_digit_composite,100.00,94.20,68.12,89.86
3_digit_easy_composite,92.57,76.73,85.15,61.39
4_digit_easy_composite,87.00,53.40,92.00,48.20
3_digit_medium_composite,89.29,32.14,94.64,37.50
4_digit_medium_composite,32.20,7.20,81.80,11.40
4_digit_hard_composite,41.98,7.63,81.68,8.40


In [19]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()

composite = 0
wrong_composite = 0

for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    wrong_patton_acc = np.array([0, 0, 0, 0])
    
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
            composite += 1

        wrong_pattons = [
                        f"{row['Number']} is divisible by 3",
                        f"{row['Number']} is divisible by 7",
                        f"{row['Number']} is divisible by 11",
                        ]
        
        for wrong_patton in wrong_pattons:
            if (row['Number']%3== 0) or (row['Number']%11== 0) or (row['Number']%7== 0):
                continue
            if wrong_patton in row['Output']:
                wrong_patton_acc[i] += 1
                wrong_composite += 1
                break
        
        if row['Output'].rfind("is divisible by") != -1:
            cut_output = row['Output'][row['Output'].rfind("is divisible by")+len("is divisible by")+1:]
            cut_output = cut_output.split(".")[0]
            cut_output = cut_output.replace("and", ",")
            cut_output = cut_output.replace(" , ", "")
            try: 
                num_list = list(map(int, cut_output.split(",")))
            except:
                # print(cut_output, row['Output'])
                pass 

            for factor in num_list:
                # replace 
                if (row['Number']%factor)==0:
                    continue
                else:
                    # replace
                    wrong_composite += 1
                    break 
            
            
            
    
    acc = acc / (len(temp_df) // 4) * 100
    wrong_patton_acc = wrong_patton_acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
    acc_df.loc["[wrong rate] "+Difficulty] = wrong_patton_acc

print(composite, "/8488")
print(wrong_composite, "/", composite)
acc_df.round(2)

3272 /8488
2047 / 3272


,Q1,Q2,Q3,Q4
2_digit_composite,100.00,94.20,68.12,89.86
[wrong rate] 2_digit_composite,0.00,2.90,0.00,1.45
3_digit_easy_composite,92.57,76.73,85.15,61.39
[wrong rate] 3_digit_easy_composite,0.00,1.98,0.00,2.48
3_digit_medium_composite,89.29,32.14,94.64,37.50
[wrong rate] 3_digit_medium_composite,0.00,8.93,0.00,7.14
4_digit_easy_composite,87.00,53.40,92.00,48.20
[wrong rate] 4_digit_easy_composite,0.00,5.60,0.00,2.80
4_digit_hard_composite,41.98,7.63,81.68,8.40
[wrong rate] 4_digit_hard_composite,0.00,9.92,0.00,2.29
